## Import all Libraries

In [1]:
# AWS Rekognition to get bbox
import numpy as np
import boto3
from PIL import Image
from matplotlib import pyplot as plt
from utils.rekognition import determine_color, draw_animal_count
import cv2
import time
import math
import os
import io
import json
import pandas as pd
from utils.config import *
from utils.fix_annotation import *
from collections import namedtuple
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

In [2]:
# process whole image.py to get key points
import mmcv
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint
import torch as tr
#from torchvision import transforms
from mmpose.apis import (inference, inference_top_down_pose_model, init_pose_model,
                         vis_pose_result)
from mmpose.models import build_posenet
from mmpose.datasets.pipelines import Compose

## Get Bounding Boxes from Video Frames

In [3]:
class LoadImage:
    """A simple pipeline to load image."""

    def __init__(self, color_type='color', channel_order='rgb'):
        self.color_type = color_type
        self.channel_order = channel_order

    def __call__(self, results):
        """Call function to load images into results.
        Args:
            results (dict): A result dict contains the img_or_path.
        Returns:
            dict: ``results`` will be returned containing loaded image.
        """
        if isinstance(results['img_or_path'], str):
            results['image_file'] = results['img_or_path']
            img = mmcv.imread(results['img_or_path'], self.color_type,
                              self.channel_order)
        elif isinstance(results['img_or_path'], np.ndarray):
            results['image_file'] = ''
            if self.color_type == 'color' and self.channel_order == 'rgb':
                img = cv2.cvtColor(results['img_or_path'], cv2.COLOR_BGR2RGB)
        else:
            raise TypeError('"img_or_path" must be a numpy array or a str or '
                            'a pathlib.Path object')
        results['img'] = img
        return results

In [4]:
def init_pose_model(config, checkpoint=None, device='cuda:0'):
    """Initialize a pose model from config file.
    Args:
        config (str or :obj:`mmcv.Config`): Config file path or the config
            object.
        checkpoint (str, optional): Checkpoint path. If left as None, the model
            will not load any weights.
    Returns:
        nn.Module: The constructed detector.
    """
    if isinstance(config, str):
        config = mmcv.Config.fromfile(config)
    elif not isinstance(config, mmcv.Config):
        raise TypeError('config must be a filename or Config object, '
                        f'but got {type(config)}')
    config.model.pretrained = None
    model = build_posenet(config.model)
    if checkpoint is not None:
        # load model checkpoint
        load_checkpoint(model, checkpoint, map_location=device)
    # save the config in the model for convenience
    model.cfg = config
    model.to(device)
    model.eval()
    return model

In [5]:
def _box2cs(cfg, box):
    """This encodes bbox(x,y,w,h) into (center, scale)
    Args:
        x, y, w, h
    Returns:
        tuple: A tuple containing center and scale.
        - np.ndarray[float32](2,): Center of the bbox (x, y).
        - np.ndarray[float32](2,): Scale of the bbox w & h.
    """

    x, y, w, h = box[:4]
    input_size = cfg.data_cfg['image_size']
    aspect_ratio = input_size[0] / input_size[1]
    center = np.array([x + w * 0.5, y + h * 0.5], dtype=np.float32)

    if w > aspect_ratio * h:
        h = w * 1.0 / aspect_ratio
    elif w < aspect_ratio * h:
        w = h * aspect_ratio

    # pixel std is 200.0
    scale = np.array([w / 200.0, h / 200.0], dtype=np.float32)

    scale = scale * 1.25

    return center, scale

In [6]:
def process_model(model, dataset, person_results, img_or_path):
    bboxes = np.array([box['bbox'] for box in person_results])
    cfg = model.cfg
    flip_pairs = None
    device = next(model.parameters()).device
    channel_order = cfg.test_pipeline[0].get('channel_order', 'rgb')
    test_pipeline = [LoadImage(channel_order=channel_order)] + cfg.test_pipeline[1:]
    test_pipeline = Compose(test_pipeline)
    if dataset == 'AnimalHorse10Dataset':
        flip_pairs = []
    else:
        raise NotImplementedError()
    batch_data = []
    for bbox in bboxes:
        center, scale = _box2cs(cfg, bbox)
        # prepare data
        data = {
            'img_or_path':
            img_or_path,
            'center':
            center,
            'scale':
            scale,
            'bbox_score':
            bbox[4] if len(bbox) == 5 else 1,
            'bbox_id':
            0,  # need to be assigned if batch_size > 1
            'dataset':
            dataset,
            'joints_3d':
            np.zeros((cfg.data_cfg.num_joints, 3), dtype=np.float32),
            'joints_3d_visible':
            np.zeros((cfg.data_cfg.num_joints, 3), dtype=np.float32),
            'rotation':
            0,
            'ann_info': {
                'image_size': np.array(cfg.data_cfg['image_size']),
                'num_joints': cfg.data_cfg['num_joints'],
                'flip_pairs': flip_pairs
            }
        }
        data = test_pipeline(data)
        batch_data.append(data)
    batch_data = collate(batch_data, samples_per_gpu=1)
    if next(model.parameters()).is_cuda:
        # scatter not work so just move image to cuda device
        batch_data['img'] = batch_data['img'].to(device)
    # get all img_metas of each bounding box
    batch_data['img_metas'] = [
        img_metas[0] for img_metas in batch_data['img_metas'].data]

    with tr.no_grad():
        result = model(
            img=batch_data['img'],
            #img = torch_data,
            img_metas=batch_data['img_metas'],
            return_loss=False,
            return_heatmap=False)
    return result['preds'], result['output_heatmap']

In [7]:
device = tr.device("cuda:0" if tr.cuda.is_available() else "cpu")
print(device)
model_head = init_pose_model(config='../my_configs/cattle/resnet_50_10/head_black.py', checkpoint='../train_result/cattle/resnet_50_10/head_black/best.pth', device = device)
model_spine = init_pose_model(config='../my_configs/cattle/resnet_50_5/spine_black.py', checkpoint='../train_result/cattle/resnet_50_5/spine_black/best.pth', device = device)
model_tail = init_pose_model(config='../my_configs/cattle/resnet_50_5/tail_black.py', checkpoint='../train_result/cattle/resnet_50_5/tail_black/best.pth', device = device)
model_leg_front = init_pose_model(config='../my_configs/cattle/resnet_50_10/leg_front_black.py', checkpoint='../train_result/cattle/resnet_50_10/leg_front_black/best.pth', device = device)
model_leg_back = init_pose_model(config='../my_configs/cattle/resnet_50_5/leg_back_black.py', checkpoint='../train_result/cattle/resnet_50_5/leg_back_black/best.pth', device = device)

dataset_head = model_head.cfg.data['test']['type']
dataset_spine = model_spine.cfg.data['test']['type']
dataset_tail = model_tail.cfg.data['test']['type']
dataset_leg_front = model_leg_front.cfg.data['test']['type']
dataset_leg_back = model_leg_back.cfg.data['test']['type']

cuda:0
Use load_from_local loader
Use load_from_local loader
Use load_from_local loader
Use load_from_local loader
Use load_from_local loader


In [8]:
def get_kp_color(label):
    # BGR
    color = (0, 0, 255)
    if label == 'Head':
        color = ['#EC51F8', '#74F54B', 
                 '#EC51F8', '#74F54B',
                 '#4394F9', '#F49736',
                 '#F49736', '#FFFB56',
                 '#FFFB56', '#4394F9',
                 '#07178D']
    elif label == 'Spine':
          color = ['#4394F9', '#4394F9', '#4394F9',
                  '#4394F9', '#4394F9', '#4394F9', 
                  '#4394F9', '#4394F9', '#24518D']
    elif label == 'Tail':
        color = ['#EC51F8', '#EC51F8',
                '#EC51F8', '#EC51F8',
                '#EC51F8', '#892B8E']
    elif label == 'Leg_front':
        color = ['#F49736', '#F49736',
                '#F49736', '#F49736',
                '#F49736', '#F49736',
                '#F49736', '#F49736',
                '#F49736', '#8C551E']
    elif label == 'Leg_back':
        color = ['#74F54B', '#74F54B',
                '#74F54B', '#74F54B',
                '#74F54B', '#74F54B',
                '#74F54B', '#74F54B',
                '#74F54B', '#3F8D28',]
    return color

In [9]:
def get_skeleton(label):
    skeleton_list = []
    if label == 'Head':
        skeleton_list = [[4, 0], [4, 2], [0, 2], [1, 3], 
                        [5, 6], [7, 8], [0, 1], [1, 5],
                        [5, 7], [7, 9], [2, 3], [3, 6],
                        [6, 8], [8, 9], [4, 9]]
    elif label == 'Spine':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4],
                        [4, 5], [5, 6], [6, 7]]
    elif label == 'Tail':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4]]
    elif label == 'Leg_front':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4],
                        [4, 5], [5, 6], [6, 7], [7, 8]]
    elif label == 'Leg_back':
        skeleton_list = [[0, 1], [1, 2], [2, 3], [3, 4],
                        [4, 5], [5, 6], [6, 7], [7, 8]]
    return skeleton_list

In [10]:
def get_kp_name(label, i):
    name_list = []
    if label == 'Head':
        name_list = ["Ear_Tip_Ri", "Ear_Base_R", "Ear_Tip_Le", "Ear_Base_L", 
                     "Head_Top", "Eye_Right", "Eye_Left", "Nostril_Ri", 
                     "Nostril_Le", "Mouth_Bott"]
    elif label == 'Spine':
        name_list = ["Head", "Neck", "Shoulder F", "Arch 1", 
                     "Arch Mid", "Arch 3", "Should Bac", "Tail"]
    elif label == 'Tail':
        name_list = ["Tail Start", "Tail 1/4", "Tail half", "Tail 3/4", "Tail End"]
    elif label == 'Leg_front':
        name_list = ["Right Hoof", "Right Ankl", "Right Knee", "Right Hip", 
                     "Shoulder", "Left Hip", "Left Knee", "Left Ankle", 
                     "Left Hoof"]
    elif label == 'Leg_back':
        name_list = ["Right Hoof", "Right Ankle", "Right Knee", "Right Hip", 
                     "Shoulder", "Left Hip", "Left Knee", "Left Ankle", 
                     "Left Hoof"]
    return name_list[i]

In [11]:
def rgb_to_bgr(color):
    color = list(color)
    temp_r = color[0]
    color[0] = color[2]
    color[2] = temp_r
    return tuple(color)

In [12]:
def get_angle(a, b, c):
    ang = math.degrees(math.atan2(
        c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    
    return ang + 360 if ang < 0 else ang


In [13]:
def hex_to_bgr(color_str):
    # convert color
    hex_color = color_str.lstrip("#")
    # convert outline color
    # bgr: 4, 2, 0 or rgb: 0, 2, 4
    return tuple(int(hex_color[i:i+2], 16) for i in (4, 2, 0))

In [14]:
def get_shift(radius, angle):
    x = radius * math.cos(math.radians(angle))
    y = radius * math.sin(math.radians(angle))
    return x, y

In [15]:
def get_bbox_type(label):
    bbox_type = 'head'
    if label == 'Spine' or label == 'Leg_front' or label == 'Leg_back':
        bbox_type = 'cow'
    elif label == 'Tail':
        bbox_type = 'tail'
    return bbox_type

In [16]:
def get_all_angle(point_1, point_2, point_3, filp):
    angle = 360 - get_angle(point_1, point_2, point_3)
    # pieslice bbox
    delta_x = point_3[0] - point_2[0]
    delta_y = point_3[1] - point_2[1]
    if filp:
        # return to original angle
        angle = -1 * angle + 360
        delta_x = point_1[0] - point_2[0]
        delta_y = point_1[1] - point_2[1]
    start_angle = math.degrees(math.atan2(delta_y, delta_x))
    end_angle = start_angle + round(angle)
    mid_angle = start_angle + round(angle) / 2
    return angle, start_angle, end_angle, mid_angle

In [17]:
def draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, fill_color, broder_color, fill_opacity, opacity_adjust=0, radius=40):
    # draw filled angle
    shapes = np.zeros_like(img, np.uint8)
#     cv2.ellipse(shapes, (int(x+shift_list[0]), int(y+shift_list[1])), (radius, radius), 0, int(angle_list[1]), int(angle_list[2]), fill_color, cv2.FILLED)
    cv2.ellipse(shapes, (int(x + shift_list[0]), int(y + shift_list[1])), 
                (radius, radius), 0, angle_list[1], angle_list[2], 
                fill_color, cv2.FILLED)
    mask = shapes.astype(bool)
    img[mask] = cv2.addWeighted(img, 1 - fill_opacity + opacity_adjust, 
                                shapes, fill_opacity, 0)[mask]
    # edge
    cv2.ellipse(img, (int(x+shift_list[0]), int(y + shift_list[1])), (radius, radius), 0, 
                angle_list[1], angle_list[2], broder_color, 2)
    # draw start angle outline
    start_point = (int(x + shift_list[0]), int(y + shift_list[1]))
    end_point = (int(x + shift_list[0] + radius * math.cos(math.radians(angle_list[1]))),
                 int(y + shift_list[1] + radius * math.sin(math.radians(angle_list[1]))))
    cv2.line(img, start_point, end_point, broder_color, 2)
    # draw end angle outline
    end_point = (int(x + shift_list[0] + radius * math.cos(math.radians(angle_list[2]))),
                 int(y + shift_list[1] + radius * math.sin(math.radians(angle_list[2]))))
    cv2.line(img, start_point, end_point, broder_color, 2)
    # draw full angle
    angle_str = "{:.0f}".format(angle_list[0])
    angle_str_len = len(angle_str)
    cv2.putText(img, angle_str, (int(x+shift_list[0])+shift_text_list[0], int(y)+shift_text_list[1]), 
                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.ellipse(img, (int(x+shift_list[0]+shift_text_list[0]+angle_str_len*12), int(y-8)+shift_text_list[1]), 
                (3, 3), 0, 0, 360, (255, 255, 255), 1)

In [18]:
def vis_pose(points, img, label, frameId, data_list, CS_THR=0.4):
    # keypoints
    kp_color = get_kp_color(label)
    # connect line
    skeleton_list = get_skeleton(label)
    for ske in skeleton_list:
        fir_pt_x, fir_pt_y, fir_pt_p = points[ske[0]]
        sec_pt_x, sec_pt_y, sec_pt_p = points[ske[1]]
        if fir_pt_p > CS_THR and sec_pt_p > CS_THR:
            # draw line
            start_point = (int(fir_pt_x), int(fir_pt_y))
            end_point = (int(sec_pt_x), int(sec_pt_y))
            bgr_color = hex_to_bgr(kp_color[-1])
            cv2.line(img, start_point, end_point, bgr_color, 5)
    
    for i, point in enumerate(points):
        x, y, p = point
        if p > CS_THR:
            x = int(x)
            y = int(y)
            bgr_color = hex_to_bgr(kp_color[i])
            # draw kp
            cv2.ellipse(img, (x, y), (5, 5), 0, 0, 360, bgr_color, cv2.FILLED)
            # draw kp outline
            bgr_color = hex_to_bgr(kp_color[-1])
            cv2.ellipse(img, (x, y), (7, 7), 0, 0, 360, bgr_color, 2)
#             cv2.putText(img, str(i), (x-20, y-20), 
#                         cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 
#                         1, cv2.LINE_AA)
            #***init keypoint data
#             bbox_type = get_bbox_type(label)
            kp_data_list = [frameId, get_kp_name(label, i), x, y, 0]
            #***
            # draw spine angle
            if (label == 'Spine') and i > 0 and i < 7 and points[i-1][2] > CS_THR and points[i+1][2] > CS_THR:
                delta_x = points[i-1][0] - points[i][0]
                angle, start_angle, end_angle, mid_angle = get_all_angle(points[i+1], points[i], points[i-1], delta_x > 0)
                kp_data_list[-1] = angle
                shift_text_list = [-20, -22]
                shift_x, shift_y = get_shift(10, mid_angle)
                shift_list = [shift_x, shift_y]
                angle_list = [angle, start_angle, end_angle, mid_angle]
                draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                  hex_to_bgr('#4394F9'), hex_to_bgr('#4394F9'), 0.8)
            # draw ear angle
            elif (label == 'Head'):
                # draw angle for point 1 of head
                if i == 1 and points[0][2] > CS_THR and points[1][2] > CS_THR and points[3][2] > CS_THR:
                    # cacluate the angle of opposite side
                    delta_x = points[3][0] - points[1][0]
                    angle, start_angle, end_angle, mid_angle = get_all_angle(points[0], points[1], points[3], delta_x > 0)
                    kp_data_list[-1] = angle
                    shift_text_list = [-15, -15]
                    shift_x, shift_y = get_shift(10, mid_angle)
                    shift_list = [shift_x, shift_y]
                    angle_list = [angle, start_angle, end_angle, mid_angle]
                    draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                      hex_to_bgr('#4394F9'), hex_to_bgr('#4394F9'), 0.8)
                # draw angle for point 3 of head
                if i == 3 and points[1][2] > CS_THR and points[2][2] > CS_THR and points[3][2] > CS_THR:
                    # cacluate the angle of opposite side
                    delta_x = points[2][0] - points[3][0]
                    angle, start_angle, end_angle, mid_angle = get_all_angle(points[1], points[3], points[2], delta_x > 0)
                    kp_data_list[-1] = angle
                    shift_text_list = [-15, -15]
                    shift_x, shift_y = get_shift(10, mid_angle)
                    shift_list = [shift_x, shift_y]
                    angle_list = [angle, start_angle, end_angle, mid_angle]
                    draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                      hex_to_bgr('#4394F9'), hex_to_bgr('#4394F9'), 0.8)
            # draw tail angle
            elif (label == 'Tail') and i > 0 and i < 4 and points[i-1][2] > CS_THR and points[i+1][2] > CS_THR and points[0][2] > CS_THR and points[1][2] > CS_THR:
                filp = (points[0][0] - points[1][0] > 0)
                angle, start_angle, end_angle, mid_angle = get_all_angle(points[i+1], points[i], points[i-1], filp)
                kp_data_list[-1] = angle
                shift_text_list = [0, 5]
                shift_x, shift_y = get_shift(10, mid_angle)
                shift_list = [shift_x, shift_y]
                angle_list = [angle, start_angle, end_angle, mid_angle]
                if filp:
                    shift_text_list[0] = -40
                draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                  hex_to_bgr('#EC51F8'), hex_to_bgr('#EC51F8'), 0.8)
            # draw leg angle
            elif (label == 'Leg_front' or label == 'Leg_back') and i > 0 and i < 8 and points[i-1][2] > CS_THR and points[i+1][2] > CS_THR:
                filp = not (i >= 5 or (i == 4 and points[5][0] < points[3][0]))
                angle, start_angle, end_angle, mid_angle = get_all_angle(points[i+1], points[i], points[i-1], filp)
                kp_data_list[-1] = angle
                shift_text_list = [0, 5]
                shift_dis = 10
                if i == 4:
                    shift_text_list[0] = 12
                    shift_dis = 15
                if i == 1 or i == 7:
                    shift_text_list[1] = 0
                shift_x, shift_y = get_shift(shift_dis, mid_angle)
                shift_list = [shift_x, shift_y]
                angle_list = [angle, start_angle, end_angle, mid_angle]
                draw_filled_angle(img, x, y, angle_list, shift_list, shift_text_list, 
                                      hex_to_bgr('#FF9300'), hex_to_bgr('#FF9300'), 0.8, opacity_adjust=0.2)
                
            data_list.append(kp_data_list)
    return img

In [19]:
def vis_mutual_angle(pred_1, pred_2, label_1, label_2, img, frameId, data_list, CS_THR=0.4):
    pred_1_idx = 4
    if label_1 == 'Head' and label_2 == 'Spine':
        x1, y1, p1 = pred_1[pred_1_idx]
        x2, y2, p2 = pred_2[0]
        x3, y3, p3 = pred_1[9]
        x4, y4, p4 = pred_2[1]
        color_code = '#4394F9'
    elif label_1 == 'Tail' and label_2 == 'Spine':
        pred_1_idx = 0
        x1, y1, p1 = pred_1[pred_1_idx]
        x2, y2, p2 = pred_2[7]
        x3, y3, p3 = pred_1[1]
        x4, y4, p4 = pred_2[6]
        color_code = '#EC51F8'
    else:
        print('no matched label')
        return img

    if p3 <= CS_THR or p4 <= CS_THR or (p1 <= CS_THR and p2 <= CS_THR):
        return img
    if p1 > CS_THR and p2 <= CS_THR:
        x2, y2, p2 = x1, y1, p1
    elif p2 > CS_THR and p1 <= CS_THR:
        x1, y1, p1 = x2, y2, p2
    bbox_type_1 = get_bbox_type(label_1)
    bbox_type_2 = get_bbox_type(label_2)
    mid_point_x = int((x1 + x2) / 2)
    mid_point_y = int((y1 + y2) / 2)
    # draw angle
    filp = (x4-x2 > 0)
    angle, start_angle, end_angle, mid_angle = get_all_angle([x3, y3], [mid_point_x, mid_point_y], [x4, y4], filp)
    kp_data_list = [frameId, get_kp_name(label_1, pred_1_idx), mid_point_x, mid_point_y, 0]
    kp_data_list[-1] = angle
    shift_text_list = [5, -8]
    shift_x, shift_y = get_shift(10, mid_angle)
    shift_list = [shift_x, shift_y]
    angle_list = [angle, start_angle, end_angle, mid_angle]
    if filp:
        shift_text_list[0] = -25
    draw_filled_angle(img, mid_point_x, mid_point_y, angle_list, shift_list, shift_text_list, 
                      hex_to_bgr(color_code), hex_to_bgr(color_code), 0.8)

    data_list.append(kp_data_list)
    return img

In [20]:
def extend_bbox(left, top, width, height, extend_rate, img_size):
    temp_left = left - (width * extend_rate / 2)
    if temp_left < 0:
        temp_left = 0
    temp_top = top - (height * extend_rate / 2)
    if temp_top < 0:
        temp_top = 0
    temp_width = width + (width * extend_rate / 2)
#     if temp_left+temp_width > img_size[0]-1:
#         temp_width = img_size[0] - 1 - temp_left
    temp_height = height + (height * extend_rate / 2)
#     if temp_top+temp_height > img_size[1]-1:
#         temp_height = img_size[1] - 1 - temp_top
    #print(temp_left, temp_top, temp_width, temp_height)
    return temp_left, temp_top, temp_width, temp_height

In [21]:
def get_color(name):
    if name == 'cow' or name == 'animal':
        color = '#FF9300'
    elif name == 'head' or name == "head left" or name == "head right":
        color = '#4394F9'
    elif name == 'tag':
        color = '#00FFFF'
    elif name == 'knee':
        color = '#FFFB00'
    elif name == 'hoof':
        color = '#00F900'
    elif name == 'tail':
        color = '#FF40FF'
    elif name == 'side left' or name == 'side right':
        color = '#FF2600'
    elif name == 'udder':
        color = '#9437FF'
    elif name == 'teat':
        color = '#FF2F92'
    else:
        color = '#000000'
    return color

def get_opacity(name):
    if name == 'cow' or name == 'animal':
        opacity = 0.3
    elif name == 'tag':
        opacity = 0.3
    elif name == 'head' or name == "head Left" or name == "head Right":
        opacity = 0.45
    elif name == 'knee':
        opacity = 0.3
    elif name == 'hoof':
        opacity = 0.35
    elif name == 'tail':
        opacity = 0.3
    elif name == 'side left' or name == 'side right':
        opacity = 0.3
    elif name == 'udder':
        opacity = 0.35
    elif name == 'teat':
        opacity = 0.3
    else:
        opacity = 0.0

    return opacity

In [22]:
def get_confidence_cut_off(name):
    if name == 'cow':
        confidence = 79.4
    elif name == 'tag':
        confidence = 86.9
    elif name == 'head':
        confidence = 92.5
    elif name == 'knee':
        confidence = 78.0
    elif name == 'hoof':
        confidence = 92.9
    elif name == 'tail':
        confidence = 73.5
    elif name == 'udder':
        confidence = 35.0
    elif name == 'teat':
        confidence = 73.0
    elif name == 'animal':
        confidence = 80.0
    else:
        confidence = 80.0

    return confidence

In [23]:
# def image_resize(image, width = 256, height = 256, inter = cv2.INTER_AREA):
#     # initialize the dimensions of the image to be resized and
#     # grab the image size
#     dim = None
#     (h, w) = image.shape[:2]

#     # if both the width and height are None, then return the
#     # original image
#     if width is None and height is None:
#         return image

#     # check to see if the width is None
#     if width is None:
#         # calculate the ratio of the height and construct the
#         # dimensions
#         r = height / float(h)
#         dim = (int(w * r), height)

#     # otherwise, the height is None
#     else:
#         # calculate the ratio of the width and construct the
#         # dimensions
#         r = width / float(w)
#         dim = (width, int(h * r))
        
#     print(dim)

#     # resize the image
#     resized = cv2.resize(image, dim, interpolation = inter)

#     # return the resized image
#     return resized

In [24]:
def crop_image(image, bbox, label):
    temp_bbox = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
    
    ori_image = Image.fromarray(image, 'RGB')    
    cropped_image = ori_image.copy().crop(temp_bbox)
    # rescale images and annotations
    size_x = 256
    size_y = 256
    
    # Image.ANTIALIAS scale the cropped image (head)
    ori_crop_size = cropped_image.size
    cropped_image.thumbnail((size_x, size_y), Image.ANTIALIAS)
    # scale keypoints
    new_image = Image.new('RGB', (size_x, size_y), color = 'black')
    new_image.paste(cropped_image)
    cropped_image_size = cropped_image.size
    scale_rate = (cropped_image_size[0]/ori_crop_size[0], cropped_image_size[1]/ori_crop_size[1])
    new_image = np.array(new_image)
    
#     cropped_image = image[int(bbox[1]):int(bbox[1]+bbox[3]), int(bbox[0]):int(bbox[0]+bbox[2])]
#     cv2.imwrite('color_img.jpg', cropped_image)
# #     size_x = 256
# #     size_y = 256
# #     print(cropped_image.shape)
# #     cropped_image_size = cropped_image.shape
# #     imRes = cv2.resize(cropped_image, width=256, interpolation=cv2.INTER_AREA)
# #     cv2.imwrite('shrink_image.jpg', imRes)
#     cv2.imwrite('shrink_image.jpg', image_resize(cropped_image))
    
    return new_image, temp_bbox, cropped_image_size, scale_rate

In [25]:
def to_ori_cordinates(preds, temp_bbox, size, scale_rate):
    #print(preds, temp_bbox, size, scale_rate)
    points = preds[0]
    for i, point in enumerate(points):
        x, y, p = point
        # point out of the bound
        if x > size[0] or y > size[1]:
            points[i][2] = 0
        else:
            points[i][0] = temp_bbox[0] + x / scale_rate[0]
            points[i][1] = temp_bbox[1] + y / scale_rate[1]

    return points

In [26]:
def get_overlap_area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy

In [27]:
def draw_line_connect(pred_1, pred_2, label_1, label_2, img, CS_THR=0.4):
    if label_1 == 'Spine' and label_2 == 'Leg_front':
        # keypoint color
        kp_color = get_kp_color(label_1)
        spine_kp_idx = 1
        legf_kp_idx = 4
        fir_pt_x, fir_pt_y, fir_pt_p = pred_1[spine_kp_idx]
        sec_pt_x, sec_pt_y, sec_pt_p = pred_2[legf_kp_idx]
        if fir_pt_p > CS_THR and sec_pt_p > CS_THR:
            # draw line
            start_point = (int(fir_pt_x), int(fir_pt_y))
            end_point = (int(sec_pt_x), int(sec_pt_y))
            bgr_color = hex_to_bgr(kp_color[-1])
            cv2.line(img, start_point, end_point, bgr_color, 5)
    elif label_1 == 'Spine' and label_2 == 'Leg_back':
        # keypoint color
        kp_color = get_kp_color(label_1)
        spine_kp_idx = len(pred_1)-2
        legb_kp_idx = 4
        fir_pt_x, fir_pt_y, fir_pt_p = pred_1[spine_kp_idx]
        sec_pt_x, sec_pt_y, sec_pt_p = pred_2[legb_kp_idx]
        if fir_pt_p > CS_THR and sec_pt_p > CS_THR:
            # draw line
            start_point = (int(fir_pt_x), int(fir_pt_y))
            end_point = (int(sec_pt_x), int(sec_pt_y))
            bgr_color = hex_to_bgr(kp_color[-1])
            cv2.line(img, start_point, end_point, bgr_color, 5)
    return img

In [28]:
def save_data(pred_list, bbox_list, preds, pred_cur_idx, bbox, bbox_cur_idx, label, frameId, bbox_conf):
    for i, pred in enumerate(preds):
        pred_list.append([frameId, label, pred_cur_idx, i, pred[0], pred[1], pred[2]])
    if label == 'Spine' or label == 'Head' or label == 'Tail':
        bbox_list.append([frameId, label, bbox_cur_idx, bbox['Left'], bbox['Top'], bbox['Width'], bbox['Height'], bbox_conf])

In [29]:
#draw response
def draw_response(img, response, animal_target, frameId, data_list, pred_list, bbox_list, draw_boundary=True, draw_fill=True, draw_btn=True):
    ori_img = img.copy()
#     for customLabel in response['CustomLabels']:
#         if 'Geometry' in customLabel:
#             box = customLabel['Geometry']['BoundingBox']
#             left, top, width, height = extend_bbox(box['Left'], box['Top'], box['Width'], box['Height'], 0, ori_img.shape)
#             label = customLabel['Name'].lower()
#             conf_cut = get_confidence_cut_off(label)
#             # skip current label
#             if customLabel['Confidence'] < conf_cut:
#                 continue
#             #draw bbox
#             color = get_color(label)
# #             opacity = round(get_opacity(label) * 255)
#             opacity = get_opacity(label)
#             start_point = (int(left), int(top))
#             end_point = (int(left+width), int(top+height))
#             bgr_color =  hex_to_bgr(color)
#             if draw_fill:
#                 # Draw filled bbox
#                 # Initialize blank mask image of same dimensions for drawing the shapes
#                 shapes = np.zeros_like(img, np.uint8)
#                 cv2.rectangle(shapes, start_point, end_point, bgr_color, cv2.FILLED)
#                 mask = shapes.astype(bool)
#                 img[mask] = cv2.addWeighted(img, 1-opacity+0.15, shapes, opacity, 0)[mask]
#             if draw_boundary:
#                 img = cv2.rectangle(img, start_point, end_point, bgr_color, 3)
#             if draw_btn:
#                 end_point = (int(left)+8+len(label)*10, int(top)+20)
#                 img = cv2.rectangle(img, start_point, end_point, bgr_color, cv2.FILLED)
#                 img = cv2.putText(img, label, (int(left)+5, int(top)+15), cv2.FONT_HERSHEY_COMPLEX, 0.50, (0,0,0), 1, cv2.LINE_AA)
    
    head_pred_list, spine_pred_list, legf_pred_list, legb_pred_list, tail_pred_list = [], [], [], [], []
    head_bbox_list, spine_bbox_list, legf_bbox_list, legb_bbox_list, tail_bbox_list = [], [], [], [], []
    #keypoints
    for customLabel in response['CustomLabels']:
        if 'Geometry' in customLabel:
            box = customLabel['Geometry']['BoundingBox']
            label = customLabel['Name'].lower()
            #lower case for comparison
            conf_cut = get_confidence_cut_off(label)
            # skip current label
            if customLabel['Confidence'] < conf_cut:
                continue
        #***** Keypoints
            if label == 'head':
                extend_rate = 0.1
                np_image = np.array(ori_img)
                head_bbox = list(extend_bbox(box['Left'], box['Top'], box['Width'], box['Height'], extend_rate, ori_img.shape))
                cropped_image, temp_bbox, size, boundary = crop_image(np_image, head_bbox, label)
                head_result = []
                head_result.append({'bbox': [0, 0, 255, 255]})
                head_pred, _ = process_model(model_head, dataset_head, head_result, cropped_image)
                head_pred = to_ori_cordinates(head_pred, temp_bbox, size, boundary)
                head_pred_list.append(head_pred)
                head_bbox_list.append(head_bbox)
                # store data to list
                pred_cur_idx = len(head_pred_list)-1
                bbox_cur_idx = len(head_bbox_list)-1
                save_data(pred_list, bbox_list, head_pred, pred_cur_idx, box, bbox_cur_idx, 'Head', frameId, customLabel['Confidence'])
#                 img = vis_pose(head_pred, img, 'Head', frameId, data_list)
            elif label == 'cow' or label == 'animal':
                extend_rate = 0.05
                np_image = np.array(ori_img)
                cow_bbox = list(extend_bbox(box['Left'], box['Top'], box['Width'], box['Height'], extend_rate, ori_img.shape))
                cropped_image, temp_bbox, size, boundary = crop_image(np_image, cow_bbox, label)
                cow_result = []
                cow_result.append({'bbox': [0, 0, 255, 255]})
                # spine
                spine_pred, _ = process_model(model_spine, dataset_spine, cow_result, cropped_image)
                spine_pred = to_ori_cordinates(spine_pred, temp_bbox, size, boundary)
                spine_pred_list.append(spine_pred)
                spine_bbox_list.append(cow_bbox)
                # store data to list
                pred_cur_idx = len(spine_pred_list)-1
                bbox_cur_idx = len(spine_bbox_list)-1
                save_data(pred_list, bbox_list, spine_pred, pred_cur_idx, box, bbox_cur_idx, 'Spine', frameId, customLabel['Confidence'])
#                 for i, pred in enumerate(spine_pred):
#                     pred_list.append([frameId, 'Spine', pred_cur_idx, i, pred[0], pred[1], pred[2]])
#                 bbox_list.append([frameId, 'Spine', bbox_cur_idx, 
#                                   cow_bbox[0], cow_bbox[1], cow_bbox[2], cow_bbox[3]])
#                 img = vis_pose(spine_pred, img, 'Spine', frameId, data_list)
                # leg back
                legb_pred, _ = process_model(model_leg_back, dataset_leg_back, cow_result, cropped_image)
                legb_pred = to_ori_cordinates(legb_pred, temp_bbox, size, boundary)
                legb_pred_list.append(legb_pred)
                legb_bbox_list.append(cow_bbox)
                # store data to list
                pred_cur_idx = len(legb_pred_list)-1
                bbox_cur_idx = len(legb_bbox_list)-1
                save_data(pred_list, bbox_list, legb_pred, pred_cur_idx, box, bbox_cur_idx, 'Leg_back', frameId, customLabel['Confidence'])
#                 img = vis_pose(legb_pred, img, 'Leg_back', frameId, data_list)
                # leg front
                extend_rate = 0.1
                np_image = np.array(ori_img)
                cow_bbox = list(extend_bbox(box['Left'], box['Top'], box['Width'], box['Height'], extend_rate, ori_img.shape))
                cropped_image, temp_bbox, size, boundary = crop_image(np_image, cow_bbox, label)
                cow_result = []
                cow_result.append({'bbox': [0, 0, 255, 255]})
                legf_pred, _ = process_model(model_leg_front, dataset_leg_front, cow_result, cropped_image)
                legf_pred = to_ori_cordinates(legf_pred, temp_bbox, size, boundary)
                legf_pred_list.append(legf_pred)
                legf_bbox_list.append(cow_bbox)
                # store data to list
                pred_cur_idx = len(legf_pred_list)-1
                bbox_cur_idx = len(legf_bbox_list)-1
                save_data(pred_list, bbox_list, legf_pred, pred_cur_idx, box, bbox_cur_idx, 'Leg_front', frameId, customLabel['Confidence'])
#                 img = vis_pose(legf_pred, img, 'Leg_front', frameId, data_list)
            elif label == 'tail':
                extend_rate = 0.05
                np_image = np.array(ori_img)
                tail_bbox = list(extend_bbox(box['Left'], box['Top'], box['Width'], box['Height'], extend_rate, ori_img.shape))
                cropped_image, temp_bbox, size, boundary = crop_image(np_image, tail_bbox, label)
                tail_bound = size
                tail_result = []
                tail_result.append({'bbox': [0, 0, 255, 255]})
                tail_pred, _ = process_model(model_tail, dataset_tail, tail_result, cropped_image)
                tail_pred = to_ori_cordinates(tail_pred, temp_bbox, size, boundary)
                tail_pred_list.append(tail_pred)
                tail_bbox_list.append(tail_bbox)
                # store data to list
                pred_cur_idx = len(tail_pred_list)-1
                bbox_cur_idx = len(tail_bbox_list)-1
                save_data(pred_list, bbox_list, tail_pred, pred_cur_idx, box, bbox_cur_idx, 'Tail', frameId, customLabel['Confidence'])
#                 img = vis_pose(tail_pred, img, 'Tail', frameId, data_list)
#*****
#     print(spine_pred_list)
#     print()
#     print(pred_list)
#     print()
#     print(spine_bbox_list)
#     print()
#     print(bbox_list)
#     for i, spine_pred in enumerate(spine_pred_list):
#         for pred in spine_pred:
#             print(pred)
#     for i, tail_pred in enumerate(tail_pred_list):
#     for i, legf_pred in enumerate(legf_pred_list):
#     for i, legb_pred in enumerate(legb_pred_list):
#     for i, head_pred in enumerate(head_pred_list):

#      # draw kp and lines in indiviual bbox
#     for i, spine_pred in enumerate(spine_pred_list):
#         # draw connection line with diff part
#         img = draw_line_connect(spine_pred_list[i], legf_pred_list[i], 'Spine', 'Leg_front', img)
#         img = draw_line_connect(spine_pred_list[i], legb_pred_list[i], 'Spine', 'Leg_back', img)
#         # draw 
#         img = vis_pose(spine_pred, img, 'Spine', frameId, data_list)
#     for i, tail_pred in enumerate(tail_pred_list):
#         img = vis_pose(tail_pred, img, 'Tail', frameId, data_list)
#     for i, legf_pred in enumerate(legf_pred_list):
#         img = vis_pose(legf_pred, img, 'Leg_front', frameId, data_list)
#     for i, legb_pred in enumerate(legb_pred_list):
#         img = vis_pose(legb_pred, img, 'Leg_back', frameId, data_list)
#     for i, head_pred in enumerate(head_pred_list):
#         img = vis_pose(head_pred, img, 'Head', frameId, data_list)
            
#     # search cooresponding bbox for drawing on mutual bbox
#     for i, spine_bbox in enumerate(spine_bbox_list):
#         # find corresponding head bbox and get mutual angle
#         for j, head_bbox in enumerate(head_bbox_list):
#             ra = Rectangle(spine_bbox[0], spine_bbox[1], spine_bbox[0]+spine_bbox[2], spine_bbox[1]+spine_bbox[3])
#             rb = Rectangle(head_bbox[0], head_bbox[1], head_bbox[0]+head_bbox[2], head_bbox[1]+head_bbox[3])
#             overlap_area = get_overlap_area(ra, rb)
#             head_area = head_bbox[2] * head_bbox[3]
#             if overlap_area and head_area * 0.2 < overlap_area:
#                 img = vis_mutual_angle(head_pred_list[j], spine_pred_list[i], 'Head', 'Spine', img, frameId, data_list)
#                 break
#         # find corresponding tail bbox and get mutual angle
#         for j, tail_bbox in enumerate(tail_bbox_list):
#             ra = Rectangle(spine_bbox[0], spine_bbox[1], spine_bbox[0]+spine_bbox[2], spine_bbox[1]+spine_bbox[3])
#             rb = Rectangle(tail_bbox[0], tail_bbox[1], tail_bbox[0]+tail_bbox[2], tail_bbox[1]+tail_bbox[3])
#             overlap_area = get_overlap_area(ra, rb)
#             tail_area = tail_bbox[2] * tail_bbox[3]
#             if overlap_area and tail_area * 0.2 < overlap_area:
#                 img = vis_mutual_angle(tail_pred_list[j], spine_pred_list[i], 'Tail', 'Spine', img, frameId, data_list)
#                 break
    return img

In [30]:
def analyzeVideo(src_video, src_bbox_json, src_img_dir, output_file, video, fps=5):
    
    start = time.time()
    srcBGR = cv2.imread(src_img_dir+str(0)+'.jpg')
    height, width, channels = srcBGR.shape
    imgSize = (width, height)
    cap = cv2.VideoCapture(src_video)
    frameRate = cap.get(fps) #frame rate
    print('FrameRate:', frameRate)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(output_file, fourcc, frameRate, imgSize) 
    
    with open(src_bbox_json) as bbox_json:
        bbox_frames = json.load(bbox_json)
        data_list = []
        pred_list = []
        bbox_list = []
        for frameId, bbox_data in enumerate(bbox_frames['Frames']):
            # skip frame 350, 450, 550, 650, 750, 850, 950
            #550, 850 opposite
            frame_list = [550]
            checking = False
            if checking and frameId not in frame_list:
                continue
            # get each image frame
            srcBGR = cv2.imread(src_img_dir+str(frameId)+'.jpg')
            img = cv2.cvtColor(srcBGR, cv2.COLOR_BGR2RGB)
            inferred_frame = draw_response(img, bbox_data, 'cow', frameId, data_list, pred_list, bbox_list)
#             # uncommet this part for testing
            if checking and (frameId in frame_list):
                temp_frame = cv2.cvtColor(inferred_frame, cv2.COLOR_BGR2RGB)
                temp_image = Image.fromarray(temp_frame)
                temp_image.save('./'+'check_trail_'+str(frameId)+'_image.jpg')
            # check each 50 frame
            if frameId % 50 == 0:
                print("Finish Processing {} frame".format(frameId))
                plt.title("Frame {}".format(int(frameId)))
                plt.savefig('debug_imgs/check_{}.jpg'.format(frameId), dpi=200)
                lap = time.time()
                print('lap time: ', lap - start)
            videoWriter.write(inferred_frame)
        df = pd.DataFrame.from_records(data_list, columns=['frameId', 'keypoint_type', 'kp_x', 'kp_y', 'angle'])
        print(df.info(verbose=True))
        kp_df = pd.DataFrame.from_records(pred_list, columns=['frameId', 'label', 'bbox_idx', 'kp_idx', 'x', 'y', 'conf'])
        print(kp_df.info(verbose=True))
        bbox_df = pd.DataFrame.from_records(bbox_list, columns=['frameId', 'label', 'bbox_idx', 
                                                        'top_left_x', 'top_left_y', 'width', 'height', 'conf'])
        print(bbox_df.info(verbose=True))
#         print(bbox_df)
        kp_df.to_csv('csv_data/'+video+'_kp.csv',index=False)
        bbox_df.to_csv('csv_data/'+video+'_bbox.csv',index=False)
#         df.to_csv('csv_data/'+video+'_angle.csv',index=False)
    videoWriter.release()
    cv2.destroyAllWindows()
    bbox_json.close()
    
    #end time
    end = time.time()
    print('total time lapse', end - start)

In [31]:
# main function
video_name_list = ['cattle_multi_1']
video_format = ['.mov']
for v_idx, video in enumerate(video_name_list):
    src_video = 'video_data/input_video/'+video+video_format[v_idx]
    src_bbox_json = 'json_data_v3/'+video+'_new_bbox.json'
    src_img_dir = 'frame_img/'+video+'/'
    output_video = 'video_data/inferred_video/inferred_v3_'+video+'.mp4'
    print(output_video)
    analyzeVideo(src_video, src_bbox_json, src_img_dir, output_video, video)
    print('finished analyzing the video '+video)
    print()

video_data/inferred_video/inferred_v3_cattle_multi_1.mp4
FrameRate: 30.006466910972193
Finish Processing 0 frame
lap time:  1.659081220626831
Finish Processing 50 frame
lap time:  50.633328914642334
Finish Processing 100 frame
lap time:  104.38447761535645
Finish Processing 150 frame
lap time:  152.61625623703003
Finish Processing 200 frame
lap time:  200.45087790489197
Finish Processing 250 frame
lap time:  249.70553612709045
Finish Processing 300 frame
lap time:  307.3155417442322
Finish Processing 350 frame
lap time:  361.18591022491455
Finish Processing 400 frame
lap time:  410.1520686149597
Finish Processing 450 frame
lap time:  468.2195131778717
Finish Processing 500 frame
lap time:  532.621080160141
Finish Processing 550 frame
lap time:  595.8801877498627
Finish Processing 600 frame
lap time:  646.1605124473572
Finish Processing 650 frame
lap time:  683.5816085338593
Finish Processing 700 frame
lap time:  725.1672384738922
Finish Processing 750 frame
lap time:  771.8210167884827

In [32]:
# %%HTML
# <video width="720" height="640" controls>
#   <source src='./video_data/inferred_video/inferred_v3_IMG_4195.mp4', type="video/mp4">
# </video>